> # **Multi Class Classification Using A CNN Model**

**Importing Libraries**

In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision  
from torchvision.transforms import transforms
import pathlib
import numpy as np
from torch.autograd import Variable

**Changing Device To GPU from CPU**

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Preparing A Data Augmenter**

In [3]:
imageAugment = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])
])

**Importing And Prcessing The Data**

In [4]:
trainpath = 'E:\Multi_Class_Classification_Using_Cnn(pytorch)\dataset'
testpath  = 'E:\Multi_Class_Classification_Using_Cnn(pytorch)\testpath'

In [5]:
traindata = DataLoader(torchvision.datasets.ImageFolder(trainpath,transform= imageAugment),batch_size=8,shuffle=True)
testdata  = DataLoader(torchvision.datasets.ImageFolder(trainpath,transform= imageAugment),batch_size=8,shuffle=True)

In [44]:
root = pathlib.Path(trainpath)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['airplane', 'bus', 'cats', 'dogs']


**Creating A New Custom Cnn Model For Classification**

In [46]:
def creatModel():
    class netX64(nn.Module):
        def __init__(self):
            super().__init__()

            # ------------convolutional layers--------------

            self.conv1 = nn.Conv2d(3,8,3,padding=1)
            # size: np.floor( (160+2*1-3)/1 )+1 = 160/2 = 80 (/2 because of maxpool)

            self.conv2 = nn.Conv2d(8,16,3,padding=1)
            # size: np.floor( (80+2*1-3)/1 )+1 = 80/2 = 40 (/2 because of maxpool)

            self.conv3 = nn.Conv2d(16,32,3,padding=1)
            # size: np.floor( (40+2*1-3)/1 )+1 = 40/2 = 20 (/2 because of maxpool)

            self.conv4 = nn.Conv2d(32,64,3,padding=1)
            # size: np.floor( (20+2*1-3)/1 )+1 = 20/2 = 10 (/2 because of maxpool)

            #---------------Feed Forward layers--------------

            self.fc1 = nn.Linear(10*10*64,32)
            self.fc2 = nn.Linear(32,16)
            self.fc3 = nn.Linear(16,4)
        
        
        
         
        def forward(self,x):
            x = F.relu(F.max_pool2d(self.conv1(x),2))
            x = F.relu(F.max_pool2d(self.conv2(x),2))
            x = F.relu(F.max_pool2d(self.conv3(x),2))
            x = F.relu(F.max_pool2d(self.conv4(x),2)) 

            # Reshaping the x for linear layers
            x = x.reshape(-1,10*10*64)

            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))

            return x 

    model  = netX64()

    lossfun = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.Adam(model.parameters(),lr=.001)



    return model,lossfun,optimizer


**Visualizing The Model Details**

In [48]:
from torchsummary import summary

model,_,_ = creatModel()
summary(model.to(device),(3,160,160))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 160, 160]             224
            Conv2d-2           [-1, 16, 80, 80]           1,168
            Conv2d-3           [-1, 32, 40, 40]           4,640
            Conv2d-4           [-1, 64, 20, 20]          18,496
            Linear-5                   [-1, 32]         204,832
            Linear-6                   [-1, 16]             528
            Linear-7                    [-1, 4]              68
Total params: 229,956
Trainable params: 229,956
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.29
Forward/backward pass size (MB): 2.93
Params size (MB): 0.88
Estimated Total Size (MB): 4.10
----------------------------------------------------------------


**Creating Function Which Train and Test The Model**

In [49]:
def InitializeTheModel(printlog):

  # number of epochs
  numepochs = 40
  
  # create a new model
  net,lossfun,optimizer = creatModel()

  # send the model to the GPU
  net.to(device)

  # initialize losses
  trainLoss = torch.zeros(numepochs)
  trainAcc  = torch.zeros(numepochs)
  testLoss = torch.zeros(numepochs)
  testAcc  = torch.zeros(numepochs)



  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    net.train()
    batchLoss = []
    batchAcc  = []
    for X,y in traindata:

      # push data to GPU
      X = X.to(device)
      y = y.to(device)

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss and error from this batch
      batchLoss.append(loss.item())
      batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...
    
    trainLoss[epochi] = np.mean(batchLoss)
    trainAcc[epochi]  = 100*np.mean(batchAcc)

    net.eval()
    X,y = next(iter(testdata)) # extract X,y from test dataloader

    # push data to GPU
    X = X.to(device)
    y = y.to(device)

    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      loss = lossfun(yHat,y)
      
    # get loss and error rate from the test batch
    testLoss[epochi] = loss.item()
    testAcc[epochi]  = 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()).item()




    output_log = f'On Epoch:{epochi}.The Model Accuracy is:{trainAcc[epochi]} and the loss is:{trainLoss[epochi]}.With A Test Accuracy Of {testAcc[epochi]}.\n'

    print(output_log) if printlog else None

    


**Final Model**

In [50]:
InitializeTheModel(True)

On Epoch:0.The Model Accuracy is:18.33333396911621 and the loss is:1.4005851745605469.With A Test Accuracy Of 12.5.

On Epoch:1.The Model Accuracy is:25.0 and the loss is:1.3835759162902832.With A Test Accuracy Of 37.5.

On Epoch:2.The Model Accuracy is:39.16666793823242 and the loss is:1.3282577991485596.With A Test Accuracy Of 75.0.

On Epoch:3.The Model Accuracy is:45.0 and the loss is:1.2320623397827148.With A Test Accuracy Of 50.0.

On Epoch:4.The Model Accuracy is:50.83333206176758 and the loss is:1.1387454271316528.With A Test Accuracy Of 87.5.

On Epoch:5.The Model Accuracy is:46.66666793823242 and the loss is:1.0836602449417114.With A Test Accuracy Of 37.5.

On Epoch:6.The Model Accuracy is:54.16666793823242 and the loss is:0.9935661554336548.With A Test Accuracy Of 25.0.

On Epoch:7.The Model Accuracy is:60.83333206176758 and the loss is:0.9079269766807556.With A Test Accuracy Of 62.5.

On Epoch:8.The Model Accuracy is:60.83333206176758 and the loss is:0.8831343054771423.With